In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from functools import reduce
from typing import Dict
from io import StringIO
from multiprocessing import Pool

In [2]:
raw_exams_df = pd.read_csv('data/TCGA_Reports.csv')
raw_exams_df.head()

,patient_filename,text
0,TCGA-BP-5195.25c0b433-5557-4165-922e-2c1eac9c26f0,Date of Recelpt: Clinical Diagnosis & History:...
1,TCGA-D7-8573.b7306a47-697d-4ed3-bbe1-81d49674a8f8,"Material: 1) Material: stomach, Method of coll..."
2,TCGA-EI-7004.13591eed-30e5-47a3-91be-7a370663d2d4,page 1 / 1. copy No. 3. Examination: Histopath...
3,TCGA-EB-A82B.23E186C6-739C-4EF1-8788-79AA89C6E87A,Patient ID: Gross Description: A mass is locat...
4,TCGA-A6-3808.e1505f65-72ef-438d-a5e1-93ed8bf6635d,SPECIMEN. Right colon. CLINICAL NOTES. PRE-OP ...


In [3]:
patients_df = pd.read_excel('data/TCGA-CDR-SupplementalTableS1.xlsx')
patients_df[patients_df['type'] == 'BRCA']['bcr_patient_barcode'].unique()

array(['TCGA-3C-AAAU', 'TCGA-3C-AALI', 'TCGA-3C-AALJ', ...,
       'TCGA-XX-A89A', 'TCGA-Z7-A8R5', 'TCGA-Z7-A8R6'], dtype=object)

In [4]:
raw_exams_df['bcr_patient_barcode'] = raw_exams_df['patient_filename'].map(lambda x: x.split('.')[0])
# print(patients_df[patients_df['type'] == 'BRCA']['bcr_patient_barcode'].unique().size, exams_df['bcr_patient_barcode'].unique().size)
exams_df = pd.concat([patients_df[patients_df['type'] == 'BRCA'], raw_exams_df], axis=1, join='inner')[['bcr_patient_barcode', 'text', 'patient_filename']]

In [5]:
# nltk.download("punkt")
# nltk.download("stopwords")

In [6]:
words_counter = nltk.FreqDist([])
def f(patient):
    return nltk.word_tokenize(patient[1])
with Pool(12) as p:
    o = p.map(f, exams_df['text'].items())

for tokens in o:
    words_counter.update(tokens)

In [31]:
redundant_tokens_set = set(stopwords.words('english'))
def word_filter(acc: Dict[str, int], curr: str, words_counter: nltk.FreqDist):
    if curr.lower() in redundant_tokens_set or len(curr) < 2: #or not curr.isalpha():
        return acc
    lowercase_key = curr.lower()
    if lowercase_key in acc:
        acc[lowercase_key] += words_counter[curr]
    else:
        acc[lowercase_key] = words_counter[curr]
        
    return acc

In [32]:
redundant_tokens_set = set(stopwords.words('english'))
filtered_words_count = reduce(lambda acc, curr: word_filter(acc, curr, words_counter), words_counter, {})
filtered_words_count

{'cm': 10361,
 'lymph': 9120,
 'tumor': 8469,
 'nodes': 5512,
 'tissue': 4971,
 'margin': 4196,
 'node': 4683,
 'specimen': 4772,
 '``': 3043,
 'submitted': 3262,
 "''": 2830,
 'right': 5222,
 'left': 4733,
 'labeled': 2666,
 'carcinoma': 3248,
 'identified': 2540,
 'section': 2275,
 'measuring': 1819,
 'mass': 1906,
 'sections': 2217,
 'received': 2641,
 'soft': 1489,
 'surface': 1326,
 'consists': 1260,
 'frozen': 1618,
 'diagnosis': 2692,
 'fresh': 1255,
 'one': 1880,
 'anterior': 1395,
 'lobe': 1294,
 'posterior': 1319,
 'invasion': 2037,
 'resection': 1400,
 'negative': 2243,
 'measures': 1100,
 'dimension': 1125,
 'patient': 1169,
 'formalin': 1057,
 '0.5': 960,
 'present': 1521,
 'inked': 945,
 '1.5': 887,
 "'s": 923,
 'tan': 862,
 'grade': 1687,
 'entirely': 976,
 'portion': 922,
 'grossly': 918,
 'bisected': 825,
 'part': 1429,
 'greatest': 1027,
 'cells': 869,
 '0.3': 745,
 'c.': 790,
 'name': 780,
 'representative': 1274,
 'cassette': 1037,
 'tube': 837,
 '10': 700,
 'two': 

In [33]:
filtered_words_count['breast']

976

In [34]:
with open('data/data_bcr_clinical_data_sample.txt', 'r') as f:
	bcr_sample_df = pd.read_csv(StringIO(''.join(f.readlines()[4:]).replace('[Not Available]', '')), sep='\t')
bcr_sample_df

,PATIENT_ID,SAMPLE_ID,OTHER_SAMPLE_ID,SPECIMEN_CURRENT_WEIGHT,DAYS_TO_COLLECTION,DAYS_TO_SPECIMEN_COLLECTION,SPECIMEN_FREEZING_METHOD,INITIAL_WEIGHT,SPECIMEN_SECOND_LONGEST_DIMENSION,IS_FFPE,...,PATHOLOGY_REPORT_UUID,SAMPLE_TYPE,SAMPLE_TYPE_ID,SHORTEST_DIMENSION,TIME_BETWEEN_CLAMPING_AND_FREEZING,TIME_BETWEEN_EXCISION_AND_FREEZING,VIAL_NUMBER,ONCOTREE_CODE,CANCER_TYPE,CANCER_TYPE_DETAILED
0,TCGA-AR-A1AR,TCGA-AR-A1AR-01,5fa9998b-deff-493e-8a8e-dc2422192a48,NaN,1416.0,NaN,NaN,250.0,NaN,NO,...,747FB91B-F523-4FA0-91DD-6014EF55643D,Primary Tumor,1,NaN,NaN,NaN,A,IDC,Breast Cancer,Breast Invasive Ductal Carcinoma
1,TCGA-AR-A1AR,TCGA-AR-A1AR-10,c1e5beaa-6103-409d-bdd4-a86c0f210014,NaN,1416.0,NaN,NaN,NaN,NaN,NO,...,NaN,Blood Derived Normal,10,NaN,NaN,NaN,A,NaN,NaN,NaN
2,TCGA-BH-A1EO,TCGA-BH-A1EO-01,9d501ca9-f50d-45fc-bcc3-fee15f9771cd,NaN,4330.0,NaN,NaN,100.0,NaN,NO,...,A2B755DD-00C4-4349-8E11-C1196EBC91D3,Primary Tumor,1,NaN,NaN,NaN,A,MDLC,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma
3,TCGA-BH-A1EO,TCGA-BH-A1EO-11,fff3e5a3-4b84-4afa-bdb5-1c6b2a6626bc,NaN,4330.0,NaN,NaN,250.0,NaN,NO,...,NaN,Solid Tissue Normal,11,NaN,NaN,NaN,A,NaN,NaN,NaN
4,TCGA-BH-A1ES,TCGA-BH-A1ES-01,aa1037f4-1414-4e05-b11a-c558a23b7e62,NaN,4242.0,NaN,NaN,220.0,NaN,NO,...,9B2253AB-2C41-4E7A-9178-B690FFD9FF78,Primary Tumor,1,NaN,NaN,NaN,A,IDC,Breast Cancer,Breast Invasive Ductal Carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,TCGA-E2-A1BC,TCGA-E2-A1BC-01,83ec5738-3a27-4100-83f2-8b30ae3b2fb7,NaN,275.0,NaN,NaN,100.0,NaN,NO,...,817D4725-01FE-4148-A8FC-3431F28D5CD8,Primary Tumor,1,NaN,NaN,NaN,A,ILC,Breast Cancer,Breast Invasive Lobular Carcinoma
2281,TCGA-E2-A1BC,TCGA-E2-A1BC-10,6f95b20b-cdc6-4240-9f6e-10e199ff7f97,NaN,275.0,NaN,NaN,NaN,NaN,NO,...,NaN,Blood Derived Normal,10,NaN,NaN,NaN,A,NaN,NaN,NaN
2282,TCGA-E2-A1BC,TCGA-E2-A1BC-11,d221f2c8-5852-4392-b166-92b709000e78,NaN,275.0,NaN,NaN,260.0,NaN,NO,...,NaN,Solid Tissue Normal,11,NaN,NaN,NaN,A,NaN,NaN,NaN
2283,TCGA-E2-A1BD,TCGA-E2-A1BD-01,1ddc4c86-029f-49b5-893c-a218240d3fa4,NaN,260.0,NaN,NaN,100.0,NaN,NO,...,DF746E32-AD2F-49CB-9AA6-4F8494C16B20,Primary Tumor,1,NaN,NaN,NaN,A,IDC,Breast Cancer,Breast Invasive Ductal Carcinoma


In [35]:
unique_disease_vocab = set([word.lower().replace(',', '') for word in " ".join(bcr_sample_df['CANCER_TYPE_DETAILED'].dropna().unique()).split()]) - redundant_tokens_set
deases_word_counts = {word: filtered_words_count[word] if word in filtered_words_count else 0 for word in unique_disease_vocab}
print(bcr_sample_df['CANCER_TYPE_DETAILED'].dropna().unique())
deases_word_counts

['Breast Invasive Ductal Carcinoma'
 'Breast Mixed Ductal and Lobular Carcinoma'
 'Breast Invasive Mixed Mucinous Carcinoma'
 'Breast Invasive Lobular Carcinoma' 'Paget Disease of the Nipple'
 'Adenoid Cystic Breast Cancer' 'Invasive Breast Carcinoma'
 'Basal Cell Carcinoma' 'Metaplastic Breast Cancer'
 'Solid Papillary Carcinoma of the Breast'
 'Malignant Phyllodes Tumor of the Breast'
 'Breast Invasive Carcinoma, NOS']


{'cancer': 477,
 'mucinous': 54,
 'mixed': 89,
 'breast': 976,
 'phyllodes': 0,
 'ductal': 300,
 'invasive': 1034,
 'disease': 121,
 'paget': 8,
 'papillary': 542,
 'cell': 902,
 'metaplastic': 3,
 'lobular': 229,
 'malignant': 121,
 'solid': 262,
 'tumor': 8469,
 'cystic': 254,
 'basal': 29,
 'nipple': 217,
 'adenoid': 0,
 'nos': 89,
 'carcinoma': 3248}

In [36]:
statistics_df = pd.DataFrame({
	'word': filtered_words_count.keys(),
 	'counts': [filtered_words_count[word] for word in filtered_words_count.keys()],
  	'indicatesDisease': [word in deases_word_counts for word in filtered_words_count.keys()]
})

In [37]:
statistics_df.to_csv('word_stats.csv')

In [39]:
with open('data/data_bcr_clinical_data_patient 1.txt', 'r') as f:
	bcr_patient_df = pd.read_csv(StringIO(''.join(f.readlines()[4:]).replace('[Not Available]', '').replace('[Not Applicable]', '')), sep='\t')
bcr_patient_df.columns.values

array(['OTHER_PATIENT_ID', 'PATIENT_ID', 'FORM_COMPLETION_DATE',
       'PROSPECTIVE_COLLECTION', 'RETROSPECTIVE_COLLECTION',
       'DAYS_TO_BIRTH', 'GENDER', 'MENOPAUSE_STATUS', 'RACE', 'ETHNICITY',
       'HISTORY_OTHER_MALIGNANCY', 'HISTORY_NEOADJUVANT_TRTYN',
       'TUMOR_STATUS', 'VITAL_STATUS', 'DAYS_TO_LAST_FOLLOWUP',
       'DAYS_TO_DEATH', 'RADIATION_TREATMENT_ADJUVANT',
       'PHARMACEUTICAL_TX_ADJUVANT', 'HISTOLOGICAL_SUBTYPE',
       'INITIAL_PATHOLOGIC_DX_YEAR', 'AGE',
       'METHOD_OF_INITIAL_SAMPLE_PROCUREMENT',
       'METHOD_OF_INITIAL_SAMPLE_PROCUREMENT_OTHER',
       'SURGICAL_PROCEDURE_FIRST', 'FIRST_SURGICAL_PROCEDURE_OTHER',
       'PATH_MARGIN', 'SURGERY_FOR_POSITIVE_MARGINS',
       'SURGERY_FOR_POSITIVE_MARGINS_OTHER', 'MARGIN_STATUS_REEXCISION',
       'STAGING_SYSTEM', 'STAGING_SYSTEM_OTHER',
       'MICROMET_DETECTION_BY_IHC', 'LYMPH_NODES_EXAMINED',
       'LYMPH_NODE_EXAMINED_COUNT', 'LYMPH_NODES_EXAMINED_HE_COUNT',
       'LYMPH_NODES_EXAMINED_IHC_COU

In [29]:
exams_df

,bcr_patient_barcode,bcr_patient_barcode,text,patient_filename
504,TCGA-3C-AAAU,TCGA-HT-8111,Tumor: Normal: Addendum Diagnosis: Anaplastic ...,TCGA-HT-8111.9de92ba0-f9d6-4f7b-8a92-548082b49288
505,TCGA-3C-AALI,TCGA-06-6389,CLINICAL HISTORY. Brain tumor. OPERATIVE DIAGN...,TCGA-06-6389.8ca0ae5b-4448-4f8b-8cdc-50e6209ced26
506,TCGA-3C-AALJ,TCGA-73-A9RS,SPECIMENS: A. 4R LYMPH NODE. B. ADDITIONAL 4R ...,TCGA-73-A9RS.7E029394-80EC-4B53-829E-8F1DA476D648
507,TCGA-3C-AALK,TCGA-10-0935,(A) RIGHT TUBE AND OVARY: HIGH-GRADE PAPILLARY...,TCGA-10-0935.f9111190-c01e-4fe7-a1db-be0434f1365f
508,TCGA-4H-AAAK,TCGA-SR-A6MS,Specimen: Procedure: Medical Record #: Accessi...,TCGA-SR-A6MS.A37F899E-D942-455B-B90F-CE52A66894E9
...,...,...,...,...
1596,TCGA-WT-AB44,TCGA-CN-A49A,Report Type. Date of Event. M. Authored by. Ho...,TCGA-CN-A49A.38A7951F-DDA7-4F2E-BF50-976459D3048C
1597,TCGA-XX-A899,TCGA-A8-A09K,Diagnosis: 4. Resected material from'l left br...,TCGA-A8-A09K.56C24ABB-CF1D-447F-A5A8-B762C4B01566
1598,TCGA-XX-A89A,TCGA-N7-A4Y0,Service : Gynecology. Gender: F. MRN. Physicia...,TCGA-N7-A4Y0.04D78BF5-445D-4F04-90ED-4AC0C8995915
1599,TCGA-Z7-A8R5,TCGA-RM-A68T,Microscopic Discussion: Sections demonstrate a...,TCGA-RM-A68T.49F14C96-CA57-469E-9615-FED1F902ED95


In [30]:
bcr_patient_df['OS_STATUS'].unique()

array(['LIVING', 'DECEASED'], dtype=object)